$$
\large \textbf{Problema 02}
$$

Considere el entramado mostrado en la figura, con una fuerza aplicada de $20\:kN$. Calcule los desplazamientos de cada uno de los nodos y las tensiones que sufre cada elemento. Todos los elementos tienen $E=210\:GPa$ y una sección de $10\:cm^2$, excepto el elemento 3, que tiene una sección de $20\:cm^2$. Los elementos 2 y 5 tienen una longitud de $8\:m$ y el elemento 3 de $4\:m$.
<center><img src="ImgProb02.png"></center>

***

In [1]:
import mef
import numpy as np

## Datos del problema
Grados de libertad por nodo:

In [2]:
glxn = 2

Matriz de nodos:

In [3]:
MN = np.array([[0, 0, 0],
               [8, 0, 0],
               [16, 0, 0],
               [8, 4, 0]])
Nn = MN.shape[0]

Matriz de conectividad:

In [4]:
MC = np.array([[0, 3],
               [0, 1],
               [1, 3],
               [2, 3],
               [1, 2]])
Ne, Nnxe = MC.shape

Módulo elástico y área de cada elemento:

In [5]:
E = 2.1E11*np.ones(5)
A = np.array([10, 10, 20, 10, 10])*1E-4

Condiciones de vínculo:

In [6]:
s = np.array([0, 1, 5])
Us = np.array([0, 0, 0]).reshape([-1, 1])

r = np.array([i for i in range(glxn*Nn) if i not in s])
Fr = np.array([0, -20000, 0, 0, 0]).reshape([-1, 1])

## Resolución
Armado de la matriz K:

In [7]:
def Kelemental(MN, MC, Ee, Ae, e):
    """
    INPUTS:
      MN = Matriz de nodos
      MC = Matriz de conectividad
      Ee = Módulo elástico del elemento
      Ae = Sección del elemento
      e  = Número de elemento
    OUTPUTS:
      Ke = Matriz K elemental
    """
    Lx = MN[MC[e, 1], 0]-MN[MC[e, 0], 0]
    Ly = MN[MC[e, 1], 1]-MN[MC[e, 0], 1]
    L = np.sqrt(Lx**2+Ly**2)
    phi = np.arctan2(Ly, Lx)
    cos = np.cos(phi)
    sin = np.sin(phi)
    Ke = (Ee*Ae/L)*np.array([[cos**2, cos*sin, -cos**2, -cos*sin],
                             [cos*sin, sin**2, -cos*sin, -sin**2],
                             [-cos**2, -cos*sin, cos**2, cos*sin],
                             [-cos*sin, -sin**2, cos*sin, sin**2]])
    Ke[np.abs(Ke/Ke.max()) < 1E-15] = 0
    return Ke

In [8]:
def Kglobal(MN, MC, E, A, glxn):
    """
    INPUTS:
      MN   = Matriz de nodos
      MC   = Matriz de conectividad
      E    = Vector de módulos elásticos de cada elemento
      A    = Vector de secciones de cada elemento
      glxn = Grados de libertad por nodo
    OUTPUTS:
      Kg = Matriz K global
    """

    Ke = {}
    file1 = 'Ke.txt'
    with open(file1, 'w') as f:
        f.write('Matrices elementales\n=================================================')
    file2 = 'Kg.txt'
    with open(file2, 'w') as f:
        f.write('Matriz global\n=================================================')

    Nn = MN.shape[0]
    Ne, Nnxe = MC.shape
    Kg = np.zeros([glxn*Nn, glxn*Nn])
    for e in range(Ne):
        Ee = E[e]
        Ae = A[e]

        Ke = Kelemental(MN, MC, Ee, Ae, e)

        fe = np.abs(Ke.max())
        with open(file1, 'a') as f:
            f.write(f'\nMatriz elemental {e}, fe ={fe:4e}\n')
            f.write(f'{Ke/fe}\n')

        for i in range(Nnxe):
            rangoi = np.linspace(i*glxn, (i+1)*glxn-1, Nnxe).astype(int)
            rangoni = np.linspace(MC[e, i]*glxn, (MC[e, i]+1)*glxn-1, Nnxe).astype(int)
            for j in range(Nnxe):
                rangoj = np.linspace(j*glxn, (j+1)*glxn-1, Nnxe).astype(int)
                rangonj = np.linspace(MC[e, j]*glxn, (MC[e, j]+1)*glxn-1, Nnxe).astype(int)
                Kg[np.ix_(rangoni, rangonj)] += Ke[np.ix_(rangoi, rangoj)]

    fe = np.abs(Kg.max())
    with open(file2, 'a') as f:
        f.write(f'\nMatriz global, fe ={fe:4e}\n')
        f.write(f'{Kg/fe}\n')
    return Kg

In [9]:
K = Kglobal(MN, MC, E, A, glxn)

Solución del problema:

In [10]:
F, U = mef.solve(K, r, Fr, s, Us)

In [11]:
for nodo in range(Nn):
    print('\033[1m \033[33m    Nodo %s \033[0m   Ux = %2.3f mm     Uy = %6.3f mm     Fx = %3.1f kN     Fy = %5.1f kN' % (nodo+1, U[2*nodo]*1000, U[2*nodo+1]*1000, F[2*nodo]/1000, F[2*nodo+1]/1000))

     Nodo 1    Ux = 0.000 mm     Uy =  0.000 mm     Fx = 0.0 kN     Fy =  10.0 kN
     Nodo 2    Ux = 0.762 mm     Uy = -3.844 mm     Fx = 0.0 kN     Fy = -20.0 kN
     Nodo 3    Ux = 1.524 mm     Uy =  0.000 mm     Fx = 0.0 kN     Fy =  10.0 kN
     Nodo 4    Ux = 0.762 mm     Uy = -3.653 mm     Fx = 0.0 kN     Fy =   0.0 kN
